<a href="https://colab.research.google.com/github/manycoding/signatures-detection/blob/master/Signatures_mmdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# %cd mmdetection

In [2]:
#@title
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -q -r requirements/build.txt
!pip install -q "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"
!pip install -q -e .

Cloning into 'mmdetection'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 6616 (delta 0), reused 0 (delta 0), pack-reused 6615
Receiving objects: 100% (6616/6616), 5.46 MiB | 13.75 MiB/s, done.
Resolving deltas: 100% (4525/4525), done.
/content/mmdetection
     |████████████████████████████████| 51kB 2.6MB/s 


In [4]:
gpu = !nvidia-smi
gpu = 0.5 if "Tesla P4" in gpu[7] else 1
gpu

1

In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import gc
import random
import os
import numpy as np
import pandas as pd
import torch
import json
from PIL import Image
from functools import partial
from pathlib import Path

root_dir = Path("/content/gdrive/My Drive/")
home = root_dir/"fastai/Signature/"
home.mkdir(parents=True, exist_ok=True)
data = home/"dataset"

In [0]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

# Data

In [0]:
raw_labels = pd.read_csv(data/"image_object_detection_1.csv",
                         names="set,image_path,label,x1,y1,x2,y1_,x2_,y2,x1_,y2_".split(","),
                         usecols=[0,1,2,8,3,4,5])[["set", "image_path", "label", "y2", "x1", "y1", "x2"]]
raw_labels["image_path"] = raw_labels["image_path"].apply(lambda x: x.split("/")[-1])

In [0]:
def process(f, labels):
    img = Image.open(f)
    labels.loc[labels["image_path"] == f.name, ["width"]] = img.size[0]
    labels.loc[labels["image_path"] == f.name, ["height"]] = img.size[1]
    labels.loc[labels["image_path"] == f.name, ["x1", "x2"]] = labels.loc[labels["image_path"] == f.name, ["x1", "x2"]] * img.size[0]
    labels.loc[labels["image_path"] == f.name, ["y2", "y1"]] = labels.loc[labels["image_path"] == f.name, ["y2", "y1"]] * img.size[1]

raw_labels["width"] = 0
raw_labels["height"] = 0

for f in raw_labels.image_path.unique():
    process(Path(data/"train"/f), raw_labels)

In [0]:
raw_labels["bboxes"] = raw_labels[["x1", "y1", "x2", "y2"]].apply(lambda x: [x[0], x[1], x[2], x[3]], axis=1)

In [0]:
labels_df = pd.DataFrame()

for n, g in raw_labels.groupby("image_path"):
    labels_df = labels_df.append([[n, g["bboxes"].to_list(), g["label"].to_list(),
                                   g["width"].to_list()[0], g["height"].to_list()[0],
                                   g["set"].to_list()[0]]], ignore_index=True)

labels_df = labels_df.rename(columns={0: "image", 1: "bboxes", 2: "labels", 3: "width", 4: "height",
                                      5: "set"})

In [12]:
labels_df.head()

,image,bboxes,labels,width,height,set
0,000001-2020-02-14T16:50:17.554Z.jpg,"[[25.416036, 294.21112000000005, 190.62027, 33...",[sign],600,400,TRAIN
1,000002-2020-02-14T16:50:17.555Z.jpg,"[[31.337625956000004, 345.40296125000003, 242....",[sign],674,425,TRAIN
2,000004-2020-02-14T16:50:17.555Z.jpg,"[[46.84218076, 398.2769637, 265.8152692, 454.1...",[sign],776,497,TRAIN
3,000005-2020-02-14T16:50:17.555Z.jpg,"[[289.838, 375.25879799999996, 604.41828, 447....",[sign],800,519,TRAIN
4,000006-2020-02-14T16:50:17.555Z.jpg,"[[577.06458, 572.474454, 690.841482, 608.16117...","[sign, sign]",1140,655,TRAIN


In [0]:
!mkdir -p data/signatures/
!ln -s "{data}/train" data/signatures/
!ln -s "{data}/test" data/signatures/

In [0]:
def create_labels(df, valid=False):
    labels_l = np.array([])
    if valid:
        _df = df[df.set != "TRAIN"]
    else:
        _df = df[df.set == "TRAIN"]
    for i, r in _df.iterrows():
        l = {"filename": r["image"],
            "width": r["width"],
            "height": r["height"],
            "ann": {
                "bboxes": np.array(r["bboxes"], dtype=np.float32),
                # "labels": r["labels"]
                "labels": np.array([0] * len(r["labels"]), dtype=np.int64)
            }}
        labels_l = np.append(labels_l, l)
    return labels_l

In [0]:
import pickle

In [49]:
create_labels(labels_df)[0]

{'ann': {'bboxes': array([[ 25.416037, 294.21112 , 190.62027 , 336.8899  ]], dtype=float32),
  'labels': array([0])},
 'filename': '000001-2020-02-14T16:50:17.554Z.jpg',
 'height': 400,
 'width': 600}

In [0]:
with open("data/signatures/train_labels.pkl", "wb") as f:
    pickle.dump(create_labels(labels_df), f)

with open("data/signatures/valid_labels.pkl", "wb") as f:
    pickle.dump(create_labels(labels_df, valid=True), f)

In [0]:
!rm -rf /content/mmdetection/work_dirs/retinanet_r50_fpn_fp16_1x

In [0]:
# from mmdet.datasets.registry import DATASETS
# from mmdet.datasets.custom import CustomDataset

# @DATASETS.register_module
# class SignDataset(CustomDataset):
#     CLASSES = ("sign")

In [58]:
!python tools/train.py "{home}/retinanet_fp16.py"

2020-02-27 15:21:13,600 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.6.9 (default, Nov  7 2019, 10:44:02) [GCC 8.3.0]
CUDA available: True
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 10.0, V10.0.130
GPU 0: Tesla T4
GCC: gcc (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0
PyTorch: 1.4.0
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - Intel(R) Math Kernel Library Version 2019.0.4 Product Build 20190411 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v0.21.1 (Git Hash 7d2fd500bc78936d1d648ca713b901012f470dbc)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CUDA Runtime 10.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_37,code=compute_3

In [59]:
!python tools/test.py "{home}/retinanet_fp16.py" {checkpoint_file} --eval mAP

[>>] 14/14, 7.2 task/s, elapsed: 2s, ETA:     0s
+-------+-----+------+--------+-------+
| class | gts | dets | recall | ap    |
+-------+-----+------+--------+-------+
| sign  | 0   | 0    | 0.000  | 0.000 |
+-------+-----+------+--------+-------+
| mAP   |     |      |        | 0.000 |
+-------+-----+------+--------+-------+


In [60]:
l = "/content/mmdetection/work_dirs/retinanet_r50_fpn_fp16_1x/20200227_150859.log.json"
!python tools/analyze_logs.py plot_curve {l} --keys loss_cls loss_reg bbox_mAP

Traceback (most recent call last):
  File "tools/analyze_logs.py", line 181, in <module>
    main()
  File "tools/analyze_logs.py", line 175, in main
    log_dicts = load_json_logs(json_logs)
  File "tools/analyze_logs.py", line 154, in load_json_logs
    with open(json_log, 'r') as log_file:
FileNotFoundError: [Errno 2] No such file or directory: '/content/mmdetection/work_dirs/retinanet_r50_fpn_fp16_1x/20200227_150859.log.json'


In [0]:
# %load_ext autoreload
# %autoreload 2

In [0]:
from mmdet.apis import init_detector, inference_detector, show_result
import mmcv

config_file = '/content/gdrive/My Drive/fastai/Signature/retinanet_fp16.py'
checkpoint_file = '/content/mmdetection/work_dirs/retinanet_r50_fpn_fp16_1x/latest.pth'

In [61]:
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')
# test a single image and show the results
model.CLASSES = ["sign"]
for fn in os.listdir(str(data/"test")):
    img = str(data/f"test/{fn}")
    # or img = mmcv.imread(img), which will only load it once
    result = inference_detector(model, img)
    # visualize the results in a new window
    display(result)
    if len(result[0]) > 0:
        show_result(img, result, model.CLASSES)

[array([], shape=(0, 5), dtype=float32)]

[array([], shape=(0, 5), dtype=float32)]

[array([], shape=(0, 5), dtype=float32)]

[array([], shape=(0, 5), dtype=float32)]

[array([], shape=(0, 5), dtype=float32)]

...

[array([], shape=(0, 5), dtype=float32)]

[array([], shape=(0, 5), dtype=float32)]

[array([], shape=(0, 5), dtype=float32)]

[array([], shape=(0, 5), dtype=float32)]

[array([], shape=(0, 5), dtype=float32)]